In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("results/all_results.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.drop(['Game', 'Date'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [ ]:
transformed_df.head()

In [ ]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

In [ ]:
# Amount of games we need to take into consideration for prediction
window_length = 20
window_length 

In [ ]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

In [ ]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
X

In [ ]:
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
y

In [ ]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[0]

In [ ]:
X[1]

In [ ]:
y[0]

In [ ]:
y[1]

In [ ]:
# Recurrent Neural Network (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 200

In [ ]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(59))
# Adding the last output layer
model.add(Dense(number_of_features))

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [ ]:
model.fit(x=X, y=y, batch_size=200, epochs=500, verbose=2)

In [ ]:
to_predict = df.tail(8)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

In [ ]:
to_predict = np.array(to_predict)
to_predict

In [ ]:
scaled_to_predict = scaler.transform(to_predict)

In [ ]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

In [ ]:
prediction = df.tail(1)
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])